In [1]:
import sys
sys.path.append('../personal_pkgs/')

import pandas as pd
import oop_func as func
import personal_pkg as ref
import parsing_func as pars
import os
import re
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

#### 저널 별 데이터를 모으자

In [2]:
fm_path = pars.journal_txt_ls('한국재무관리학회')
sc_path = pars.journal_txt_ls('한국증권학회지')
dr_path = pars.journal_txt_ls('한국파생상품학회')
f_path = pars.journal_txt_ls('한국재무학회')

In [4]:
fm_sent_ls , fm_error_ls = pars.split_to_sent(fm_path)
f_sent_ls , f_error_ls = pars.split_to_sent(f_path)
sc_sent_ls , sc_error_ls = pars.split_to_sent(sc_path)
dr_sent_ls , dr_error_ls = pars.split_to_sent(dr_path)

0
100
200
300
400
0
100
200
0
100
200
300
400
500
0
100
200


In [14]:
len(fm_sent_ls) , len(f_sent_ls) , len(sc_sent_ls) , len(dr_sent_ls)

(12562, 9174, 12307, 8661)

In [15]:
len(fm_error_ls) , len(f_error_ls) , len(sc_error_ls) , len(dr_error_ls)

(76, 3, 24, 58)

위의 Error term 을 Fix 하는 코드를 추가 삽입한다.

In [ ]:
fm_error_path_ls = [fm_error_ls[idx][1] for idx in range(len(fm_error_ls))]
f_error_path_ls = [f_error_ls[idx][1] for idx in range(len(f_error_ls))]
sc_error_path_ls = [sc_error_ls[idx][1] for idx in range(len(sc_error_ls))]
dr_error_path_ls = [dr_error_ls[idx][1] for idx in range(len(dr_error_ls))]

vol23_no3_213.txt 는 무슨 방법을 써도 안되더라..

In [ ]:
set([i[1].split("/")[-1] for i in fm_error_ls + f_error_ls + sc_error_ls + dr_error_ls]).difference(os.listdir('error_solved_file/'))

In [ ]:
print('한국재무관리연구')
fm_fix_error_ls = pars.fix_the_error_file(fm_error_path_ls)
print('한국재무학회')
f_fix_error_ls = pars.fix_the_error_file(f_error_path_ls)
print('한국증권학회지')
sc_fix_error_ls = pars.fix_the_error_file(sc_error_path_ls)
print('한국파생상품학회')
dr_fix_error_ls = pars.fix_the_error_file(dr_error_path_ls)

잘 나와있는 것도 있지만, 대부분 이미지 파일이기 때문에 깨져 있는 경우가 많다. 아래는 재무관리연구학회에서 에러가 났던 파일을 소생시켜 파싱한 리스트이다.

In [ ]:
fm_sent_ls += fm_fix_error_ls
f_sent_ls += f_fix_error_ls
sc_sent_ls += sc_fix_error_ls
dr_sent_ls += dr_fix_error_ls

In [ ]:
len(fm_sent_ls) , len(f_sent_ls) , len(sc_sent_ls) , len(dr_sent_ls)

- `*_q_ls` : double quotation 이 있는 sentence. 분석 가능한 문장이라고 간주해 dictionary 를 형성하는 데 사용된다.
- `*_nq_ls` : quotation 이 없는 sentence. 불안정한 문장, 즉, 딕셔너리를 형성하는데에는 부적합하다고 간주한다.

In [ ]:
fm_q_ls , fm_nq_ls = pars.show_quote_and_unquote(fm_sent_ls)
f_q_ls , f_nq_ls = pars.show_quote_and_unquote(f_sent_ls)
sc_q_ls , sc_nq_ls = pars.show_quote_and_unquote(sc_sent_ls)
dr_q_ls , dr_nq_ls = pars.show_quote_and_unquote(dr_sent_ls)

In [ ]:
fm_ip_ls = pars.make_imperfect_ls(fm_nq_ls)
f_ip_ls = pars.make_imperfect_ls(f_nq_ls)
sc_ip_ls = pars.make_imperfect_ls(sc_nq_ls)
dr_ip_ls = pars.make_imperfect_ls(dr_nq_ls)

#### 딕셔너리 만들기

In [ ]:
fm_df,unique_fm_df = pars.make_dictionary(fm_q_ls)
f_df,unique_f_df = pars.make_dictionary(f_q_ls)
sc_df,unique_sc_df = pars.make_dictionary(sc_q_ls)
dr_df,unique_dr_df = pars.make_dictionary(dr_q_ls)

In [14]:
fm_df['from'] = ['한국재무관리학회' for _ in range(len(fm_df))]
f_df['from'] = ['한국재무학회' for _ in range(len(f_df))]
sc_df['from'] = ['한국증권학회' for _ in range(len(sc_df))]
dr_df['from'] = ['한국파생상품학회' for _ in range(len(dr_df))]

In [15]:
dictionary_df = pd.concat([fm_df,f_df,sc_df,dr_df],ignore_index=True)
dictionary_df.shape

(22701, 6)

#### 점수 기반해서 quotation 이 없는 sentence 에 대해서 논문 이름을 추출한다

In [16]:
idx_ls = []
print(dictionary_df.shape)

for idx,val in enumerate(dictionary_df['paper'].tolist()) :
    if val not in [i[1] for i in idx_ls] :
        idx_ls.append((idx,val))

quote_df = dictionary_df.iloc[[i[0] for i in idx_ls]]
quote_df.reset_index(inplace=True)
print(quote_df.shape )

(22701, 6)
(16995, 7)


아래의 데이터프레임을 보면, `‘` 하나만 있는 reference 가 있다. 분석 결과, similarity score기반에서 너무 많이 catch up 하는 현상이 발생하였다. 이를 의도적으로 제거해준다.

In [17]:
quote_df[quote_df['paper'] == '‘']

,index,author,year,paper,journal,raw,from
15428,20311,"19’‘”에 '""… ‘ N. o. ’ . '"" s‘· ’……… _""_ .A Cι...",None,‘,"”에 '""“… “‘ N. o. ’ .“ '"" “s‘·“ ’……… _""_ .A C...","19’“‘”에 '""“… “‘ N. o. ’ .“ '"" “s‘·“ ’……… _""_...",한국파생상품학회


In [18]:
quote_df = quote_df[quote_df['paper'] != '‘']

In [19]:
quote_df.reset_index(drop=True,inplace=True)
quote_df.tail()

,index,author,year,paper,journal,raw,from
16989,22695,"정진호, 임병진, 원종현,",2002,"국채선물을 이용한 적정 헤지비율 추정에 관한 연구,","” 「증권학회지, | 30, ,","정진호, 임병진, 원종현, “국채선물을 이용한 적정 헤지비율 추정에 관한 연구,” ...",한국파생상품학회
16990,22696,"홍정효, 문규현,",2004,"원·달러 선물시장을 이용한 헤지효과성,","” 「재무관리연구」, 21, 1, , ,","홍정효, 문규현, “원·달러 선물시장을 이용한 헤지효과성,” 「재무관리연구」, 21...",한국파생상품학회
16991,22698,"홍정효,",2009,"동태적 헤지모형을 이용한 유로화 선물시장의 헤지성과 분석,","” 「금융공학연구」, | 8, 1, ,","홍정효, “동태적 헤지모형을 이용한 유로화 선물시장의 헤지성과 분석,” 「금융공학연...",한국파생상품학회
16992,22699,"홍정효, 문규현,",2010,"개별주식선물시장의 헤지성과에 관한 실증적 연구 : 정태적헤지모형 vs ,| 동태적헤...","” 「재무관리연구」, 27, 3, ,","홍정효, 문규현, “개별주식선물시장의 헤지성과에 관한 실증적 연구 : 정태적헤지모형...",한국파생상품학회
16993,22700,",23 ,김인준이 동행 ,<부록 1> 헤지전략을 고려하지 않은 투자 전략에 대한 ,...",2011,환율과 주가 간 상간관계는..... -0.50 이상의 강한 음(-)의 상관관계를 보...,"” 보건복지부 년 11월 24일 보도자료, p.2. ,25 ,김인준. 이 동행 ,<...",",23 ,김인준이 동행 ,<부록 1> 헤지전략을 고려하지 않은 투자 전략에 대한 ,...",한국파생상품학회


아래의 데이터프레임을 보면, author 와 journal이 missclassificate되었지만, paper의 이름이 `results` 라는 이유로, similarity score에서 너무 많이 catch up 된다. 이에 따라, 의도적으로 제거해준다.

In [20]:
quote_df[quote_df['paper'] == 'results']

,index,author,year,paper,journal,raw,from
13951,17906,Cifuentes A. and G. Katsaros 2007 The One-Fact...,2007,results,” Journal of Structured Finance 9 Fall. Cox D....,Cifuentes A. and G. Katsaros 2007 The One-Fact...,한국증권학회


In [21]:
quote_df = quote_df[quote_df['paper'] != 'results']

In [22]:
quote_df.reset_index(drop=True,inplace=True)
quote_df.tail()

,index,author,year,paper,journal,raw,from
16988,22695,"정진호, 임병진, 원종현,",2002,"국채선물을 이용한 적정 헤지비율 추정에 관한 연구,","” 「증권학회지, | 30, ,","정진호, 임병진, 원종현, “국채선물을 이용한 적정 헤지비율 추정에 관한 연구,” ...",한국파생상품학회
16989,22696,"홍정효, 문규현,",2004,"원·달러 선물시장을 이용한 헤지효과성,","” 「재무관리연구」, 21, 1, , ,","홍정효, 문규현, “원·달러 선물시장을 이용한 헤지효과성,” 「재무관리연구」, 21...",한국파생상품학회
16990,22698,"홍정효,",2009,"동태적 헤지모형을 이용한 유로화 선물시장의 헤지성과 분석,","” 「금융공학연구」, | 8, 1, ,","홍정효, “동태적 헤지모형을 이용한 유로화 선물시장의 헤지성과 분석,” 「금융공학연...",한국파생상품학회
16991,22699,"홍정효, 문규현,",2010,"개별주식선물시장의 헤지성과에 관한 실증적 연구 : 정태적헤지모형 vs ,| 동태적헤...","” 「재무관리연구」, 27, 3, ,","홍정효, 문규현, “개별주식선물시장의 헤지성과에 관한 실증적 연구 : 정태적헤지모형...",한국파생상품학회
16992,22700,",23 ,김인준이 동행 ,<부록 1> 헤지전략을 고려하지 않은 투자 전략에 대한 ,...",2011,환율과 주가 간 상간관계는..... -0.50 이상의 강한 음(-)의 상관관계를 보...,"” 보건복지부 년 11월 24일 보도자료, p.2. ,25 ,김인준. 이 동행 ,<...",",23 ,김인준이 동행 ,<부록 1> 헤지전략을 고려하지 않은 투자 전략에 대한 ,...",한국파생상품학회


#### 0.9 라는 similarity score 를 기준으로 삼아서, double quoatation 이 없는 문장의 paper name 을 catch 해준다.

In [23]:
import concurrent.futures

ip_ls = [fm_ip_ls,
          f_ip_ls ,
          sc_ip_ls ,
          dr_ip_ls]
cat_ls = list(zip([quote_df for i in range(len(ip_ls))],ip_ls))
len(cat_ls)

4

In [24]:
def main() : 
    final_df = []
    with concurrent.futures.ProcessPoolExecutor(max_workers = 8) as executor:
        for _,df_ in zip(cat_ls,executor.map(pars.catch_the_paper_under_similarity_score_for_concurrent, cat_ls)):
            final_df.append(df_)
    return final_df

if __name__ == '__main__':
    %time final_df = main()

the length of dictionary is 16993.. the cost of function is proportional with it
0
the length of dictionary is 16993.. the cost of function is proportional with it
0
0
the length of dictionary is 16993.. the cost of function is proportional with it
the length of dictionary is 16993.. the cost of function is proportional with it
0
4000
8000
4000
4000
12000
16000
8000
8000
4000
12000
12000
16000
8000
16000
12000
16000
CPU times: user 195 ms, sys: 92.9 ms, total: 288 ms
Wall time: 41min 34s


In [25]:
len(final_df)

4

In [26]:
fm_catch_paper = final_df[0]
f_catch_paper = final_df[1]
sc_catch_paper = final_df[2]
dr_catch_paper =final_df[3]

In [27]:
print(len(fm_catch_paper),
len(f_catch_paper),
len(sc_catch_paper),
len(dr_catch_paper))

457 122 4189 550


총합친 것도 필요하지만, 우리에게 필요한 것은 저널별 데이터이다.

In [28]:
fm_detect_ls = [i[0] for i in fm_catch_paper]
f_detect_ls = [i[0] for i in f_catch_paper]
sc_detect_ls = [i[0] for i in sc_catch_paper]
dr_detect_ls = [i[0] for i in dr_catch_paper]

In [29]:
fm_paper_name_ls = dictionary_df[dictionary_df['from'] == '한국재무관리학회']['paper'].tolist()
fm_paper_name_ls = fm_paper_name_ls + fm_detect_ls
len(fm_paper_name_ls)

10813

In [30]:
f_paper_name_ls = dictionary_df[dictionary_df['from'] == '한국재무학회']['paper'].tolist()
f_paper_name_ls = f_paper_name_ls + f_detect_ls
len(f_paper_name_ls)

6913

In [31]:
sc_paper_name_ls = dictionary_df[dictionary_df['from'] == '한국증권학회']['paper'].tolist()
sc_paper_name_ls = sc_paper_name_ls + sc_detect_ls
len(sc_paper_name_ls)

7272

In [32]:
dr_paper_name_ls = dictionary_df[dictionary_df['from'] == '한국파생상품학회']['paper'].tolist()
dr_paper_name_ls = dr_paper_name_ls + dr_detect_ls
len(dr_paper_name_ls)

3021

In [33]:
fm_final_df = pd.DataFrame(columns=['author','year','paper','journal'])

for idx in range(len(fm_paper_name_ls)) : 
    if len(fm_final_df) % 4000 == 0 : print(len(fm_final_df))
    append_df = quote_df[quote_df['paper'] == fm_paper_name_ls[idx]]
    fm_final_df = pd.concat([fm_final_df,append_df])

0
4000
8000


In [34]:
fm_final_df.to_csv('fm_base_df.csv',index=False)

In [35]:
f_final_df = pd.DataFrame(columns=['author','year','paper','journal'])

for idx in range(len(f_paper_name_ls)) : 
    if len(f_final_df) % 4000 == 0 : print(len(f_final_df))
    append_df = quote_df[quote_df['paper'] == f_paper_name_ls[idx]]
    f_final_df = pd.concat([f_final_df,append_df])

0
4000


In [36]:
f_final_df.to_csv('f_base_df.csv',index=False)

In [37]:
sc_final_df = pd.DataFrame(columns=['author','year','paper','journal'])

for idx in range(len(sc_paper_name_ls)) : 
    if len(sc_final_df) % 4000 == 0 : print(len(sc_final_df))
    append_df = quote_df[quote_df['paper'] == sc_paper_name_ls[idx]]
    sc_final_df = pd.concat([sc_final_df,append_df])

0
4000


In [38]:
sc_final_df.to_csv('sc_base_df.csv',index=False)

In [39]:
dr_final_df = pd.DataFrame(columns=['author','year','paper','journal'])

for idx in range(len(dr_paper_name_ls)) : 
    if len(dr_final_df) % 4000 == 0 : print(len(dr_final_df))
    append_df = quote_df[quote_df['paper'] == dr_paper_name_ls[idx]]
    dr_final_df = pd.concat([dr_final_df,append_df])

0


In [40]:
dr_final_df.to_csv('dr_base_df.csv',index=False)

#### catch_paper_name_ls 은 저널에 관계없이 non-quoted sentence 에서 총 어떤 논문 이름을 잡았는지에 대한 데이터를 담은 리스트이다.

In [41]:
catch_paper_name_ls = [i[0] for i in fm_catch_paper]+\
[i[0] for i in f_catch_paper]+\
[i[0] for i in sc_catch_paper]+\
[i[0] for i in dr_catch_paper]
len(catch_paper_name_ls)

5318

- final_paper_name_ls 는 paper name 으로 uniquely indexing 해주기 전의 double quotation sentence 를 담은 dataframe 이다.
- non double quotation sentence 데이터에서 추출한 catch_paper_name_ls를 dictionary_df 의 인덱스와 합치면 총 논문 이름에 대한 리스트를 얻게 된다.

In [42]:
base_paper_name_ls = dictionary_df['paper'].tolist()
final_paper_name_ls = base_paper_name_ls + catch_paper_name_ls
len(final_paper_name_ls)

28019

final_paper_name_ls 의 논문 이름 포맷은 전적으로 quote_df 를 따른다. 이에 따라서, Counting 을 할 때, quote_df 의 포맷을 유지하면서 `expanding` 시켜준다.

In [43]:
base_df = pd.DataFrame(columns=['author','year','paper','journal'])

for idx in range(len(final_paper_name_ls)) : 
    if len(base_df) % 4000 == 0 : print(len(base_df))
    append_df = quote_df[quote_df['paper'] == final_paper_name_ls[idx]]
    base_df = pd.concat([base_df,append_df])

0
4000
8000
12000
16000
20000
24000
28000


In [44]:
base_df.to_csv('base_df.csv',index=False)

- `base_df` 는 저널에 관계없이 전체 논문에 대해서 similarity score를 기반으로 최대한 merge 한 데이터들을 담은 dataframe 이다.
- `{}_base_df` 는 {} 에 붙은 alias 에 따라 특정 논문에 대한 base_df 이다.